# Работа с текстовыми данными

## Практика - шаг 0 - Подготовка данных
- загрузите из файла **movie_reviews.csv** отзывы о кинофильмах
- выведите количество положительных и отрицательных отзывов
- получите новый признак - длина отзыва
- посчитайте корреляцию длины отзыва и позитивности отзыва
- обучите логистическую регрессию и посчитайте метрику правильности

In [ ]:
import pandas as pd
import numpy as np
import string

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = pd.read_csv('movie_reviews.csv')
df.head()

,review,positive
0,"tristar / 1 : 30 / 1997 / r ( language , viole...",0
1,arlington road 1/4 . directed by mark pellingt...,0
2,the brady bunch movie is less a motion picture...,0
3,janeane garofalo in a romantic comedy -- it wa...,0
4,"i'm going to keep this plot summary brief , so...",0


In [ ]:
df['review'][0]

'tristar / 1 : 30 / 1997 / r ( language , violence , dennis rodman ) cast : jean-claude van damme ; mickey rourke ; dennis rodman ; natacha lindinger ; paul freeman director : tsui hark screenplay : dan jakoby ; paul mones ripe with explosions , mass death and really weird hairdos , tsui hark\'s " double team " must be the result of a tipsy hollywood power lunch that decided jean-claude van damme needs another notch on his bad movie-bedpost and nba superstar dennis rodman should have an acting career . actually , in " double team , " neither\'s performance is all that bad . i\'ve always been the one critic to defend van damme -- he possesses a high charisma level that some genre stars ( namely steven seagal ) never aim for ; it\'s just that he\'s never made a movie so exuberantly witty since 1994\'s " timecop . " and rodman . . . well , he\'s pretty much rodman . he\'s extremely colorful , and therefore he pretty much fits his role to a t , even if the role is that of an ex-cia weapons

In [ ]:
df['positive'].value_counts()

0    700
1    700
Name: positive, dtype: int64

In [ ]:
df['len'] = df['review'].apply(len)
df

,review,positive,len
0,"tristar / 1 : 30 / 1997 / r ( language , viole...",0,3215
1,arlington road 1/4 . directed by mark pellingt...,0,6144
2,the brady bunch movie is less a motion picture...,0,1768
3,janeane garofalo in a romantic comedy -- it wa...,0,3211
4,"i'm going to keep this plot summary brief , so...",0,6864
...,...,...,...
1395,one of the last entries in the long-running ca...,1,2663
1396,"hype ? sheesh , like no other . this side of t...",1,2206
1397,for those of us who weren't yet born when the ...,1,3032
1398,what starts out as a monotonous talking-head m...,1,3373


In [ ]:
df['positive'].corr(df['len'])

0.1296155774988609

In [ ]:
lr = LogisticRegression().fit(df.loc[:, 'len':'len'], df['positive'])
y_pred = lr.predict(df.loc[:, 'len':'len'])
accuracy_score(df['positive'], y_pred)

ValueError: ignored

In [ ]:
y_pred[y_pred == 0], y_pred[y_pred == 1]

(array([], dtype=int64), array([1, 1, 1, ..., 1, 1, 1]))

## Практика - шаг 1 - Мешок слов
- сформируйте "мешок слов" с помощью CountVectorizer
- выведите получившийся словарь и его длину
- получите матрицу признаков и посмотрите на ее представление

In [ ]:
# your code here

In [ ]:
vec = CountVectorizer()
X_vect = vec.fit_transform(df['review'])
y = df['positive']
X_vect

<1400x35590 sparse matrix of type '<class 'numpy.int64'>'
	with 473956 stored elements in Compressed Sparse Row format>

In [ ]:
X_vect.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 2, 0, ..., 0, 0, 0]])

In [ ]:
vec.vocabulary_

{'tristar': 32540,
 '30': 295,
 '1997': 232,
 'language': 17949,
 'violence': 34017,
 'dennis': 8420,
 'rodman': 26819,
 'cast': 5108,
 'jean': 16927,
 'claude': 5925,
 'van': 33684,
 'damme': 7860,
 'mickey': 20111,
 'rourke': 26983,
 'natacha': 21180,
 'lindinger': 18522,
 'paul': 23086,
 'freeman': 12631,
 'director': 8945,
 'tsui': 32638,
 'hark': 14375,
 'screenplay': 27732,
 'dan': 7877,
 'jakoby': 16851,
 'mones': 20580,
 'ripe': 26681,
 'with': 34981,
 'explosions': 11270,
 'mass': 19533,
 'death': 8049,
 'and': 1554,
 'really': 25523,
 'weird': 34568,
 'hairdos': 14179,
 'double': 9559,
 'team': 31384,
 'must': 21019,
 'be': 2954,
 'the': 31658,
 'result': 26350,
 'of': 22007,
 'tipsy': 31994,
 'hollywood': 15047,
 'power': 24227,
 'lunch': 18942,
 'that': 31653,
 'decided': 8110,
 'needs': 21292,
 'another': 1671,
 'notch': 21689,
 'on': 22122,
 'his': 14945,
 'bad': 2602,
 'movie': 20839,
 'bedpost': 3040,
 'nba': 21243,
 'superstar': 30802,
 'should': 28457,
 'have': 14491,

## Практика - шаг 2 - Сравнение моделей
Обучите модели
- логистическую регрессию
- kNN
- решающее дерево
Посчитайте для каждой модели качество на кросс-валидации

In [ ]:
# your code here

In [ ]:
lr = LogisticRegression(solver = 'liblinear')
knn = KNeighborsClassifier()
tree = DecisionTreeClassifier(max_depth=8)

print('LR', cross_val_score(lr, X_vect, y).mean())
print('KNN', cross_val_score(knn, X_vect, y).mean())
print('TREE', cross_val_score(tree, X_vect, y).mean())

LR 0.7949999999999999
KNN 0.6207142857142858
TREE 0.6242857142857143


## Практика - шаг 3 - Значимые слова
На логистической регрессии получите значимость признаков.

In [ ]:
# your code here

In [ ]:
lr = LogisticRegression(solver = 'liblinear').fit(X_vect, y)
lr.coef_[0]

array([-0.03024475, -0.09677732, -0.13205599, ...,  0.01056022,
        0.01316945,  0.00023238])

In [ ]:
vec.get_feature_names()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['00',
 '000',
 '007',
 '00749ece25ab063e882568de001da736',
 '01',
 '0195714',
 '02',
 '0215545',
 '0299',
 '03',
 '04',
 '05',
 '05pm',
 '06577663ed95e8618825695d00660958',
 '07',
 '08',
 '10',
 '100',
 '1000',
 '10021more',
 '100m',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '10s',
 '10th',
 '11',
 '110',
 '111',
 '112',
 '113',
 '1138',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '1198',
 '11th',
 '12',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '1298',
 '12th',
 '13',
 '130',
 '131',
 '132',
 '134',
 '135',
 '136',
 '137',
 '138',
 '139',
 '13th',
 '14',
 '140',
 '142',
 '143',
 '144',
 '147',
 '14a',
 '14th',
 '15',
 '150',
 '150th',
 '151',
 '153',
 '155',
 '157',
 '158',
 '1583',
 '1590',
 '15somethingsomething',
 '15th',
 '16',
 '160',
 '1600',
 '1600s',
 '161',
 '1613',
 '162',
 '164',
 '165',
 '1660',
 '1662',
 '1680',
 '16mm',
 '16th',
 '16x9',
 '17',
 '170',
 '1709',
 '171',
 '172',
 '175',
 '177',
 '1

In [ ]:
weights = sorted(zip(vec.get_feature_names_out(), lr.coef_[0]), key = lambda x: x[1])
df_weights = pd.DataFrame(weights, columns=['token', 'weight'])
df_weights

,token,weight
0,bad,-0.546157
1,director,-0.486306
2,worst,-0.422162
3,boring,-0.421594
4,anything,-0.412807
...,...,...
35585,job,0.349616
35586,fun,0.435355
35587,also,0.438250
35588,most,0.460606


## Практика - шаг 4 - улучшаем извлечение слов
Повторите шаги 1-2 с параметром **min_df** (минимальная частота слова, при котором оно учитывается).

Повторите шаги 1-2 с параметром **max_df** (максимальная частота слова, при котором оно учитывается).

Добавьте работу со стоп-словами

In [ ]:
# your code here

In [ ]:
vec = CountVectorizer(min_df = 3)
X_vect = vec.fit_transform(df['review'])
y = df['positive']
X_vect

lr = LogisticRegression(solver = 'liblinear')
knn = KNeighborsClassifier()
tree = DecisionTreeClassifier(max_depth=8)

print('LR', cross_val_score(lr, X_vect, y).mean())
print('KNN', cross_val_score(knn, X_vect, y).mean())
print('TREE', cross_val_score(tree, X_vect, y).mean())

LR 0.7949999999999999
KNN 0.6207142857142858
TREE 0.6178571428571429


In [ ]:
vec = CountVectorizer(min_df = 155)
X_vect = vec.fit_transform(df['review'])
y = df['positive']
X_vect

lr = LogisticRegression(solver = 'liblinear')
knn = KNeighborsClassifier()
tree = DecisionTreeClassifier(max_depth=8)

print('LR', cross_val_score(lr, X_vect, y).mean())
print('KNN', cross_val_score(knn, X_vect, y).mean())
print('TREE', cross_val_score(tree, X_vect, y).mean())

LR 0.7150000000000001
KNN 0.6285714285714286
TREE 0.6078571428571429


In [ ]:
vec = CountVectorizer(min_df = 3, max_df = 0.8)
X_vect = vec.fit_transform(df['review'])
y = df['positive']
X_vect

lr = LogisticRegression(solver = 'liblinear')
knn = KNeighborsClassifier()
tree = DecisionTreeClassifier(max_depth=8)

print('LR', cross_val_score(lr, X_vect, y).mean())
print('KNN', cross_val_score(knn, X_vect, y).mean())
print('TREE', cross_val_score(tree, X_vect, y).mean())

LR 0.8035714285714285
KNN 0.5785714285714286
TREE 0.615


In [ ]:
vec = CountVectorizer(min_df = 3, max_df = 0.8, stop_words='english')
X_vect = vec.fit_transform(df['review'])
y = df['positive']
X_vect

lr = LogisticRegression(solver = 'liblinear')
knn = KNeighborsClassifier()
tree = DecisionTreeClassifier(max_depth=8)

print('LR', cross_val_score(lr, X_vect, y).mean())
print('KNN', cross_val_score(knn, X_vect, y).mean())
print('TREE', cross_val_score(tree, X_vect, y).mean())

LR 0.8049999999999999
KNN 0.552142857142857
TREE 0.6157142857142858


## Практика - шаг 5 - Важность слов
Выполните ячейку ниже и проинтерпретируйте полученный результат.

Отмасштабируйте данные исходного датасета с помощью метода **tf-idf**  и повторите шаги 1-3.

Какой из  способов, опробованных выше, подойдет, чтобы улучшить полученный результат?
 Проверьте свою гипотезу.

In [ ]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vec = TfidfVectorizer()
X = vec.fit_transform(corpus)
print(vec.get_feature_names_out())

X.toarray()

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']


array([[0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524],
       [0.        , 0.6876236 , 0.        , 0.28108867, 0.        ,
        0.53864762, 0.28108867, 0.        , 0.28108867],
       [0.51184851, 0.        , 0.        , 0.26710379, 0.51184851,
        0.        , 0.26710379, 0.51184851, 0.26710379],
       [0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524]])

In [ ]:
# your code here

In [ ]:
#тут был перебор нормального человека, а потом сделал перебор на больших числах

## Практика - шаг 6 - Порядок слов
Обучите логистическую регрессию на данных с эн-грамами, выведите значимые признаки

In [ ]:
# your code here

In [ ]:
vec = CountVectorizer(min_df = 3, max_df = 0.8, stop_words='english', ngram_range= (1, 2))
X_vect = vec.fit_transform(df['review'])
y = df['positive']
X_vect

lr = LogisticRegression(solver = 'liblinear')
knn = KNeighborsClassifier()
tree = DecisionTreeClassifier(max_depth=8)

print('LR', cross_val_score(lr, X_vect, y).mean())
print('KNN', cross_val_score(knn, X_vect, y).mean())
print('TREE', cross_val_score(tree, X_vect, y).mean())

LR 0.7587465564738293
KNN 0.5091460055096418
TREE 0.6038980716253444


In [ ]:
vec = CountVectorizer(min_df = 3, max_df = 0.8, stop_words='english', ngram_range = (2, 2))
X_vect = vec.fit_transform(df['review'])
y = df['positive']

lr = LogisticRegression(solver = 'liblinear').fit(X_vect, y)
weights = sorted(zip(vec.get_feature_names(), lr.coef_[0]), key = lambda x: x[1])
df_weights = pd.DataFrame(weights, columns = ['token', 'weight'])
df_weights

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,token,weight
0,make sense,-0.805684
1,waste time,-0.664692
2,rest movie,-0.641465
3,bad movie,-0.631716
4,hour half,-0.576166
...,...,...
4247,disney studios,0.530375
4248,matt damon,0.533014
4249,story line,0.564550
4250,drew barrymore,0.584503


## Практика - шаг 7 - Оптимизация
С помощью решетчатого поиска подберите оптимальные значения моделей, обучите с ними модели, посчитайте качество и выведите значимые признаки.

In [ ]:
# your code here

In [ ]:
from sklearn.pipeline import Pipeline

vec = CountVectorizer()
lr = LogisticRegression(solver='liblinear')

pipe = Pipeline(steps = [('vec', vec), ('lr', lr)])

X = df['review']
y = df['positive']
pipe.fit(X, y)
cross_val_score(pipe, X, y).mean()

0.7487465564738293

In [ ]:
vec = CountVectorizer()
lr = LogisticRegression(solver='liblinear')

pipe = Pipeline(steps = [('vec', vec), ('lr', lr)])

params = {'vec__max_df': np.arange(0.8, 1, 0.1),
       'vec__min_df': np.arange(0.01, 0.3, 0.1),
       'vec__ngram_range': [(1, 1), (1, 2)],
       'lr__C': np.arange(0.1, 0.2, 0.1),
       }

gs = GridSearchCV(pipe, params, n_jobs = -1, verbose = 1)

X = df['review']
y = df['positive']

gs.fit(X, y)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 12 candidates, totalling 60 fits


(0.7653856749311295,
 {'lr__C': 0.1,
  'vec__max_df': 0.8,
  'vec__min_df': 0.01,
  'vec__ngram_range': (1, 1)})

## Практика - шаг 8 - Обработка естественного языка
- С помощью библиотеки **nltk** (Natural Language Toolkit) для примера текста выделите токены в виде слов, затем в виде предложений.
- Проведите нормализацию слов для примера текста

Для исходного датасета с отзывами на фильмы:
- Выделите токены-слова
- Проведите нормализацию слов
- Удалите знаки препинания и переведите все слова в нижний регистр
- Векторизуйте нормализованный текст с масштабированием
- Обучите логистическую регрессию и посчитайте метрику качества на кросс-валидации
- Выведите значимые признаки

In [ ]:
data = '''Natural language processing (NLP) is a subfield of linguistics, computer science,
and artificial intelligence concerned with the interactions between computers and
human language, in particular how to program computers to process and analyze
large amounts of natural language data. The result is a computer capable of "understanding"
the contents of documents, including the contextual nuances of the language within them.
The technology can then accurately extract information and insights contained in
the documents as well as categorize and organize the documents themselves. '''

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# your code here

In [ ]:
words = word_tokenize(data)
words

['Natural',
 'language',
 'processing',
 '(',
 'NLP',
 ')',
 'is',
 'a',
 'subfield',
 'of',
 'linguistics',
 ',',
 'computer',
 'science',
 ',',
 'and',
 'artificial',
 'intelligence',
 'concerned',
 'with',
 'the',
 'interactions',
 'between',
 'computers',
 'and',
 'human',
 'language',
 ',',
 'in',
 'particular',
 'how',
 'to',
 'program',
 'computers',
 'to',
 'process',
 'and',
 'analyze',
 'large',
 'amounts',
 'of',
 'natural',
 'language',
 'data',
 '.',
 'The',
 'result',
 'is',
 'a',
 'computer',
 'capable',
 'of',
 '``',
 'understanding',
 "''",
 'the',
 'contents',
 'of',
 'documents',
 ',',
 'including',
 'the',
 'contextual',
 'nuances',
 'of',
 'the',
 'language',
 'within',
 'them',
 '.',
 'The',
 'technology',
 'can',
 'then',
 'accurately',
 'extract',
 'information',
 'and',
 'insights',
 'contained',
 'in',
 'the',
 'documents',
 'as',
 'well',
 'as',
 'categorize',
 'and',
 'organize',
 'the',
 'documents',
 'themselves',
 '.']

In [ ]:
words =  sent_tokenize(data)
words

['Natural language processing (NLP) is a subfield of linguistics, computer science, \nand artificial intelligence concerned with the interactions between computers and \nhuman language, in particular how to program computers to process and analyze \nlarge amounts of natural language data.',
 'The result is a computer capable of "understanding" \nthe contents of documents, including the contextual nuances of the language within them.',
 'The technology can then accurately extract information and insights contained in \nthe documents as well as categorize and organize the documents themselves.']

In [ ]:
#НОРМАЛИЗАЦИЯ

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
words =  sent_tokenize(data)

stemm = SnowballStemmer('english')
s_words = [stemm.stem(w) for w in words]

z = zip(words, s_words)
pd.DataFrame(z, columns = ['original', 'Stem'])[:20]

,original,Stem
0,Natural language processing (NLP) is a subfiel...,natural language processing (nlp) is a subfiel...
1,"The result is a computer capable of ""understan...","the result is a computer capable of ""understan..."
2,The technology can then accurately extract inf...,the technology can then accurately extract inf...


In [ ]:
words =  sent_tokenize(data)

stem = SnowballStemmer('english')
s_words = [stem.stem(w) for w in words]

lemm = WordNetLemmatizer()
l_words = [lemm.lemmatize(w) for w in words]

z = zip(words, s_words, l_words)
pd.DataFrame(z, columns = ['original', 'Stem', 'Lem'])[20:40]

,original,Stem,Lem


In [ ]:
#Для примера рассмотрели работы разных способ приведения слова к начальной форме
#МЫ НЕ ЧИСТИЛИ НАБОР ДАННЫХ!!!!!!

def clean(data):

  #убраить точки, запятые, знаки...
  #привести к нижнему регистры
  #удаление окончаний

  return cleaned_data

## Практика - шаг 9 - Работа с русским языком

In [ ]:
data_rus = '''Вчера после работы заскочила в Магнит за продуктами. Передо мной на кассу стояла женщина с сынишкой лет 5. Все время, пока мы ждали своей очереди, ребенок канючил:
— Ма-а-ам, я хочу большой Киндер! Давай купим большой Киндер! Ну ма-а-ам, купи!
Уставшая от этих стонов женщина обернулась к сыну и указала на лежащий на ленте сверток.
— Нет, не куплю. Ты же видишь, мы сегодня купили конфеты!
— Ну да, да... — горестно вздохнул ребятенок, но уже в следующий момент в его глазах засветилась неугасимая надежда. — Но ведь КОГДА-НИБУДЬ ты мне его обязательно купишь, правда?!
И, окрыленный этой мыслью, он вприпрыжку поскакал за уже расплатившейся матерью, радостно улыбаясь всем вокруг.
Эх, как все-таки мало детям надо для счастья.'''

In [ ]:
stem = SnowballStemmer('russian')
stem.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

In [ ]:
w_rus = word_tokenize(data_rus)
w_rus_orig = sent_tokenize(data_rus)

stemm = SnowballStemmer('russian')
s_words = [stemm.stem(w) for w in w_rus]

z = zip(w_rus_orig, s_words)
pd.DataFrame(z, columns = ['original', 'Stem'])[:20]

,original,Stem
0,Вчера после работы заскочила в Магнит за проду...,вчер
1,Передо мной на кассу стояла женщина с сынишкой...,посл
2,"Все время, пока мы ждали своей очереди, ребено...",работ
3,Давай купим большой Киндер!,заскоч
4,"Ну ма-а-ам, купи!",в
5,Уставшая от этих стонов женщина обернулась к с...,магн
6,"— Нет, не куплю.",за
7,"Ты же видишь, мы сегодня купили конфеты!",продукт
8,"— Ну да, да... — горестно вздохнул ребятенок, ...",.
9,— Но ведь КОГДА-НИБУДЬ ты мне его обязательно ...,перед


In [ ]:
!pip install pymorphy2
import pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 47.0 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=9ac404c80ce0ba04115f3d0283dc83709192b0c75185fe68900c32f09d891258
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [ ]:
w_rus = word_tokenize(data_rus)
w_rus_orig = sent_tokenize(data_rus)
w_lem_rus = word_tokenize(data_rus)

stemm = SnowballStemmer('russian')
s_words = [stemm.stem(w) for w in w_rus]

morph = pymorphy2.MorphAnalyzer()
w_lem_rus = [morph.normal_forms(word)[0] for word in w_lem_rus]

z = zip(w_rus_orig, s_words, w_lem_rus)
pd.DataFrame(z, columns = ['original', 'Stem', 'Lem'])[:20]

,original,Stem,Lem
0,Вчера после работы заскочила в Магнит за проду...,вчер,вчера
1,Передо мной на кассу стояла женщина с сынишкой...,посл,после
2,"Все время, пока мы ждали своей очереди, ребено...",работ,работа
3,Давай купим большой Киндер!,заскоч,заскочить
4,"Ну ма-а-ам, купи!",в,в
5,Уставшая от этих стонов женщина обернулась к с...,магн,магнит
6,"— Нет, не куплю.",за,за
7,"Ты же видишь, мы сегодня купили конфеты!",продукт,продукт
8,"— Ну да, да... — горестно вздохнул ребятенок, ...",.,.
9,— Но ведь КОГДА-НИБУДЬ ты мне его обязательно ...,перед,перед


In [ ]:
pd.DataFrame(z, columns = ['original', 'Stem', 'Lem'])[20:40]

,original,Stem,Lem
